In [3]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

In [ ]:
Task 1: Mark all launch sites on a map

In [ ]:
Nhiệm vụ 1: Đánh dấu tất cả các địa điểm phóng trên bản đồ

In [4]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(url)

spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [5]:
# Select relevant sub-columns
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]


launch_sites_df = spacex_df.groupby('Launch Site', as_index=False).first()[['Launch Site', 'Lat', 'Long']]

launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [6]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)


circle = folium.Circle(
    nasa_coordinate,
    radius=1000,
    color='#d35400',
    fill=True,
    fill_opacity=0.4,
    popup=folium.Popup('NASA Johnson Space Center')
)
circle.add_to(site_map)

# Thêm icon chữ "NASA JSC"
marker = folium.map.Marker(
    nasa_coordinate,
    icon=folium.DivIcon(
        html='<div style="font-size:12px; color:#d35400;"><b>NASA JSC</b></div>'
    )
)
marker.add_to(site_map)

site_map

In [7]:

# Create a base map centered around the average coordinates
# Tạo bản đồ cơ sở tập trung xung quanh tọa độ trung bình
site_map = folium.Map(location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()], zoom_start=5)

# Add a marker for each launch site
# Thêm một điểm đánh dấu cho mỗi địa điểm phóng
for _, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(site_map)

# Display the map
site_map

In [ ]:
Task 2: Mark the success/failed launches for each site on the map

In [ ]:
Nhiệm vụ 2: Đánh dấu các lần khởi chạy thành công/thất bại cho mỗi địa điểm trên bản đồ

In [8]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [9]:
marker_cluster = MarkerCluster()

In [10]:
def marker_color(launch_outcome):
    return 'green' if launch_outcome == 1 else 'red'
site_map = folium.Map(location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()], zoom_start=5)

site_map.add_child(marker_cluster)
for index, record in spacex_df.iterrows():
    color = marker_color(record['class'])
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=f"Launch Site: {record['Launch Site']}<br>Class: {record['class']}",
        icon=folium.Icon(color='white', icon_color=color, icon='rocket', prefix='fa')
    )
    marker_cluster.add_child(marker)
site_map

In [ ]:
TASK 3: Calculate the distances between a launch site to its proximities

In [ ]:
NHIỆM VỤ 3: Tính toán khoảng cách từ một địa điểm phóng đến các khu vực lân cận

In [11]:

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)
site_map

In [12]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [13]:
# Define coordinates for launch site
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Define coordinates for coastline (move eastward toward the ocean)
coastline_lat = 28.56320
coastline_lon = -80.56000  # adjusted eastward

# Calculate distance
distance_coastline = calculate_distance(
    launch_site_lat, launch_site_lon,
    coastline_lat, coastline_lon
)

print("Distance to coastline:", distance_coastline, "km")

# Draw marker and line
folium.Marker(
    [coastline_lat, coastline_lon],
    icon=folium.Icon(color='orange'),
    popup=f"Coastline: {distance_coastline:.2f} KM"
).add_to(site_map)

folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon],
               [coastline_lat, coastline_lon]],
    color='orange'
).add_to(site_map)

site_map


Distance to coastline: 1.6431808067656966 km


In [14]:
# Create and add a folium.Marker on the selected closest coastline point on the map
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' 
             % "{:10.2f} KM".format(distance_coastline),
    ),
)
site_map.add_child(distance_marker)

# Create a folium.PolyLine between the launch site and coastline
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
lines = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

site_map